In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import os
import ast
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import gc
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_dtype(torch.float32)

In [2]:
data_root = os.path.join("./", "data/")

x_train = np.load(os.path.join(data_root, "X_train.npy"))
y_train_raw = pd.read_csv(os.path.join(data_root, "y_train.csv"), header=None)

# convert strings to corresponding arrays
y_train_raw[0] = y_train_raw[0].apply(lambda x: ast.literal_eval(x))
y_train_raw = y_train_raw[0].values

x_test = np.load(os.path.join(data_root, "X_test.npy"))
y_test_raw = pd.read_csv(os.path.join(data_root, "y_test.csv"), header=None)
y_test_raw[0] = y_test_raw[0].apply(lambda x: ast.literal_eval(x))
y_test_raw = y_test_raw[0].values

class_to_index = {
    "NORM": 0,
    "MI": 1,
    "HYP": 2,
    "STTC": 3,
    "CD": 4
}

# Encoding the labels for multi-label classification
y_test = torch.zeros((len(y_test_raw), len(class_to_index)), dtype=torch.float32)
for i, classification in enumerate(y_test_raw):
    for class_name in classification:
        y_test[i, class_to_index[class_name]] = 1

y_train = torch.zeros((len(y_train_raw), len(class_to_index)), dtype=torch.float32)
for i, classification in enumerate(y_train_raw):
    for class_name in classification:
        y_train[i, class_to_index[class_name]] = 1

x_train = torch.tensor(x_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)

# Free up some memory
del y_train_raw
del y_test_raw

In [3]:
BATCH_SIZE = 128

train_set = torch.utils.data.TensorDataset(x_train, y_train)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)

test_set = torch.utils.data.TensorDataset(x_test, y_test)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

In [27]:
class PositionalEncoding(nn.Module):
    def __init__(self, max_len=1000, emb_size=12):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_len, emb_size)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, emb_size, 2).float() * (-np.log(10000.0) / emb_size))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer("pe", pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]

class Transformer(nn.Transformer):
    def __init__(self, emb_size=12, nhead=6, depth=6, hidden_size=128, seq_length=1000, num_classes=5):
        super(Transformer, self).__init__(d_model=emb_size, nhead=nhead, num_encoder_layers=depth, num_decoder_layers=depth, dim_feedforward=hidden_size)
    
        self.pos_encoder = PositionalEncoding(seq_length, emb_size)
        self.decoder = nn.Linear(emb_size, num_classes)
        
    def forward(self, x):
        #x = self.pos_encoder(x)
        x = self.encoder(x)
        x = x.mean(dim=1)
        x = self.decoder(x)
        x = torch.sigmoid(x)
        return x
    

In [37]:
def train(net, optimizer, criterion, train_loader, epochs=10, scheduler=None):
    net = net.to(device)

    train_losses = []

    for _ in range(epochs):
        pbar = tqdm(train_loader, total=len(train_loader))
        last_i = 0
        running_loss = 0.0
        running_acc = 0.0
        for i, (x, y) in enumerate(pbar):
            x, y = x.to(device), y.to(device)

            optimizer.zero_grad()
            y_pred = net(x)
            loss = criterion(y_pred, y)
            loss.backward()
            nn.utils.clip_grad_norm_(net.parameters(), 5)
            optimizer.step()
            
            train_losses.append(loss.item())
            
            # exact match ratio
            acc = accuracy_score(y.cpu().detach().numpy(), y_pred.cpu().detach().numpy().round())
            running_loss += loss.item()
            running_acc += acc
            
            if i % 20 == 1:
                running_loss /= (i - last_i)
                running_acc /= (i - last_i)
                pbar.set_description(f"loss: {running_loss:.4f}, acc: {running_acc:.4f}")
                running_acc = 0.0
                running_loss = 0.0
                last_i = i
                
            
            if scheduler is not None:
                scheduler.step(loss.item())

    return train_losses

In [39]:
gc.collect()
torch.cuda.empty_cache()

net = Transformer(nhead=6, hidden_size=512, depth=4)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
criterion = nn.BCELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=50, verbose=True, cooldown=20, factor=0.5, min_lr=1e-6)
train(net, optimizer, criterion, train_loader, epochs=10, scheduler=scheduler)
print("done")

loss: 0.4475, acc: 0.3453:  69%|██████▉   | 107/154 [00:11<00:05,  8.88it/s]

Epoch 00260: reducing learning rate of group 0 to 5.0000e-04.


loss: 0.4460, acc: 0.3559:  36%|███▌      | 55/154 [00:06<00:11,  8.94it/s]

Epoch 00362: reducing learning rate of group 0 to 2.5000e-04.


loss: 0.4414, acc: 0.3312:  19%|█▉        | 30/154 [00:03<00:13,  8.94it/s]

...

Epoch 00930: reducing learning rate of group 0 to 1.9531e-06.


loss: 0.4390, acc: 0.3461:  51%|█████     | 78/154 [00:08<00:08,  8.85it/s]

Epoch 01001: reducing learning rate of group 0 to 1.0000e-06.


loss: 0.4288, acc: 0.3566: 100%|██████████| 154/154 [00:17<00:00,  8.89it/s]


[0.7164759039878845,
 0.6298056840896606,
 0.6172776818275452,
 0.6190471649169922,
 0.6184773445129395,
 0.5989018678665161,
 0.5665106177330017,
 0.5999088287353516,
 0.573437511920929,
 0.5800739526748657,
 0.5679255127906799,
 0.5697973966598511,
 0.5729047656059265,
 0.5494944453239441,
 0.5626229047775269,
 0.5715336799621582,
 0.5478313565254211,
 0.5643463730812073,
 0.5564497113227844,
 0.5626509785652161,
 0.5633071064949036,
 0.5651668906211853,
 0.5788410902023315,
 0.5425630807876587,
 0.5551902055740356,
 0.5536721348762512,
 0.5533437132835388,
 0.5542740821838379,
 0.5525462031364441,
 0.5881929397583008,
 0.5485308766365051,
 0.5342990159988403,
 0.5599250197410583,
 0.5330125093460083,
 0.5387675166130066,
 0.5498184561729431,
 0.5623831152915955,
 0.5277512669563293,
 0.5434938669204712,
 0.5493959784507751,
 0.538787841796875,
 0.5687098503112793,
 0.547488808631897,
 0.5507074594497681,
 0.5396263003349304,
 0.5323004126548767,
 0.5470455288887024,
 0.5552889704704